<a href="https://colab.research.google.com/github/parasgulati8/Web-Scraper/blob/master/Nashville%20Buildings%20Information%20Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/'My Drive'/'Colab Notebooks'/dataset/

/content/drive/My Drive/Colab Notebooks/dataset


In [0]:
# import glob
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# import urllib3
import requests
# import json
# from urllib.request import urlopen, urlretrieve, quote
# from openpyxl import Workbook
import re
import os

In [0]:
# Property Summary Page
def get_property_summary(index):
    print('fetching Property Summary for ', index)
    url = 'http://www.padctn.org/prc/property/' + str(index)
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    lists = soup.find_all('ul', attrs={'class':'att'})
    table = dict()
    for i in range(len(lists)):
      lists[i] = lists[i].find_all('li')
      lists[i] = [item.text for item in lists[i]]
      
      for j in range(len(lists[i])):
        colon = lists[i][j].find(':')
        lists[i][j] = [lists[i][j][:colon], lists[i][j][colon+2:]]
          
      for pair in lists[i]:
        key , value = pair
        table[key] = value
    table['Property#'] = index
    summary = pd.DataFrame(table, index=[index])
    return summary


In [0]:
# Historical data
def get_historical_data(index):
    print('fetching historical data for ', index)
    url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/1/historical'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    parcel = soup.h2.text
    parcel = ' '.join(parcel.split()[6:])
    dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                          attrs={"class":"table table-condensed table-bordered"})
    sales_history, previous_appraisals = dfs[0], dfs[1]
    sales_history['Map & Parcel'] = [parcel]*len(sales_history)
    sales_history['Property #'] = [index]*len(sales_history)
    previous_appraisals['Map & Parcel'] = [parcel]*len(previous_appraisals)
    previous_appraisals['Property #'] = [index]*len(previous_appraisals)
    return sales_history, previous_appraisals
                        

In [0]:
# Building and Improvement Details Page
def get_building_improvement(index):
    print('fetching building improvement for ', index)
    url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/1/interior'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    print('heading =',soup.h3.text)
    parcel = ''.join(re.findall(r'\d+.' , ' '.join((soup.h3.text).split())))[:-1] 
    # print('parcel = ', parcel)
    lists = (soup.find_all('ul', attrs={'class':'att'}))
    table = dict()
    for i in range(len(lists)):
      lists[i] = lists[i].find_all('li')
      lists[i] = [item.text for item in lists[i]]
      # lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
      for j in range(len(lists[i])):
        colon = lists[i][j].find(':')
        lists[i][j] = [lists[i][j][:colon], lists[i][j][colon+2:]]

      for pair in lists[i]:
        key , value = pair
        table[key] = value
    
    temp_table = pd.DataFrame(table, index=[0])
    temp_table['Property#'] = [index]
    dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text)
    building_footage, building_features = dfs[0], dfs[1]
    building_features['Property#'] = [index]*(1 if len(building_features) == 0 else len(building_features))
    building_footage['Property#'] = [index]*len(building_footage)
    merged = pd.merge(building_footage, building_features, on='Property#')
    building_improvement = pd.merge(merged, temp_table, on='Property#')
    building_improvement['Map & Parcel'] = parcel
    return building_improvement

In [0]:
def terminate(index, patience):
    for i in range(index+1,index+patience):
        temp_history, temp_appraisal = get_historical_data(i)
        if (np.any(temp_history) or np.any(temp_appraisal)):                  
            return False
    return True

# Scraping Property Summary Pages

In [0]:
i = 1
while True:
  print(i)
  _property_summary = get_property_summary(i)
  # print(_property_summary['Map & Parcel'].values)
  # print(_property_summary)

  if _property_summary['Map & Parcel'].values == '':
    try:
        end  = terminate(i, 10)
        print ('Checking to terminate')  
        if end == True:
            print('Scraping Ends. Last propert # scraped is', i-1)
            print (_property_summary)
            summary = pd.read_csv('property_summary.csv')
            summary = pd.concat((summary, property_summary))
            summary.to_csv('property_summary.csv', index_label=False)
            break
    except:
        print('Exception occured in checking termination for index', i)      
        break

  if i % 1000 == 1:
    print('Creating new dataframe')
    if np.any(_property_summary):
        property_summary = _property_summary
  else:
    if np.any(_property_summary):
        property_summary = pd.concat((property_summary, _property_summary))
    if i%1000 == 0:
      if 'property_summary.csv' in os.listdir('./'): #directory to be changed
        read = pd.read_csv('property_summary.csv')    
        read = pd.concat((read, property_summary))
        # print('Saving the data so far to csv')
        read.to_csv('property_summary.csv', index_label=False)
      else:
        if np.any(_property_summary):
            property_summary = pd.concat((property_summary, _property_summary))
        # print('Saving Initial file')
        property_summary.to_csv('property_summary.csv', index_label=False)
  i = i+1


time.sleep(5)
import boto3

ACCESS_KEY = ''
SECRET_KEY = ''
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, 
						aws_secret_access_key=SECRET_KEY)
# s3.create_bucket(Bucket="scraper-bucket",
#                  CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
files = ["Property Summary.csv"]

for i in files:
    # upload files to the bucket
    s3.upload_file(i, "v-scraper-bucket", i)

# Scraping Building History Pages

In [0]:
i = 1
while True:
  print(i)
  try:
    _sales_history, _previous_appraisals = get_historical_data(i)
    # print(_sales_history['Map & Parcel'].values)
    # print(_sales_history)

    if (not np.any(_sales_history['Map & Parcel'].values)) and (not np.any(_previous_appraisals['Map & Parcel'].values)):
      # try:
          end  = terminate(i, 10)
          print ('Checking to terminate')  
          if end == True:
              print('Scraping Ends. Last propert # scraped is', i-1)
              print (_sales_history, _previous_appraisals)

              sales = pd.read_csv('Sales History.csv')
              sales = pd.concat((sales, sales_history))
              sales.to_csv('Sales History.csv')
              apprasals = pd.read_csv('Previous Appraisals.csv')
              apprasals = pd.concat((apprasals, previous_appraisals))
              apprasals.to_csv('Previous Appraisals.csv', index_label=False)
              break
      # except:
          print('Exception occured in checking termination for index', i)      
          break
    if i % 1000 == 1:
      print('Creating new dataframe')
      if np.any(_sales_history):
        sales_history, previous_appraisals = _sales_history, _previous_appraisals
    else:
      if np.any(_sales_history):
        sales_history = pd.concat((sales_history, _sales_history))
      
      if np.any(_previous_appraisals):
        previous_appraisals = pd.concat((previous_appraisals, _previous_appraisals))
      
      if i%1000 == 0:
        # if the sales history.csv already exists, then append the data to this file
        if 'Sales History.csv' in os.listdir('./'): 
          read = pd.read_csv('Sales History.csv')    
          read = pd.concat((read, sales_history))
          print('Saving the data so far to csv')
          read.to_csv('Sales History.csv', index_label=False)
        else: # if the sales history.csv do not exists, then create a file
          print('Saving Initial file')
          sales_history.to_csv('Sales History.csv', index_label=False)

        # if the Previous Appraisals.csv already exists, then append the data to this file
        if 'Previous Appraisals.csv' in os.listdir('./'): #directory to be changed
          read = pd.read_csv('Previous Appraisals.csv')    
          read = pd.concat((read, previous_appraisals))
          print('Saving the data so far to csv')
          read.to_csv('Previous Appraisals.csv', index_label=False)
        else: # if the Previous Appraisals.csv doesn't exists, then create a new file
          print('Saving Initial file')
          previous_appraisals.to_csv('Previous Appraisals.csv', index_label=False)
  except:
    pass
  i = i+1
  time.sleep(1)



import boto3

ACCESS_KEY = ''
SECRET_KEY = ''
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, 
						aws_secret_access_key=SECRET_KEY)
# s3.create_bucket(Bucket="scraper-bucket",
#                  CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
files = ["Sales History.csv",
         "Previous Appraisals.csv"]

for i in files:
    # upload files to the bucket
    s3.upload_file(i, "v-scraper-bucket", i)

# Scrape Building Improvement Pages

In [0]:
i = 1
while True:
  print(i)
  try:
    _building_improvement = get_building_improvement(i)
    # print(_building_improvement['Map & Parcel'].values)
    # print(_building_improvement)

    if not np.any(_building_improvement['Map & Parcel'].values):
      try:
        end  = terminate(i, 10)
        print ('Checking to terminate')  
        if end == True:
            print('Scraping Ends. Last propert # scraped is', i)
            print (_building_improvement)
            improvement = pd.read_csv('Building Improvement.csv')
            improvement = pd.concat((improvement, building_improvement))
            improvement.to_csv('Building Improvement.csv', index_label=False)
            break
      except:
        print('Exception occured in checking termination for index', i)      
        break

    if i % 1000 == 1:
      # print('Creating new dataframe')
      if np.any(_building_improvement):
        building_improvement = _building_improvement
    else:
      if np.any(_building_improvement):
        building_improvement = pd.concat((building_improvement, _building_improvement))
      
      if i%1000 == 0:
        # if the building improvement.csv already exists, then append the data to this file
        if 'Building Improvement.csv' in os.listdir('./'): 
          read = pd.read_csv('Building Improvement.csv')    
          read = pd.concat((read, building_improvement))
          # print('Saving the data so far to csv')
          read.to_csv('Building Improvement.csv', index_label=False)
        else: # if the sales history.csv do not exists, then create a file
          # print('Saving Initial file')
          building_improvement.to_csv('Building Improvement.csv', index_label=False)
  except:
    pass
  i = i+1
  time.sleep(1)

#sending files to s3  
import boto3

ACCESS_KEY = ''
SECRET_KEY = ''
s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY, 
						aws_secret_access_key=SECRET_KEY)
# s3.create_bucket(Bucket="scraper-bucket",
#                  CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
files = ["Building Improvement.csv"]

for i in files:
    # upload files to the bucket
    s3.upload_file(i, "v-scraper-bucket", i)